<a href="https://colab.research.google.com/github/weiwenying/colab-notes/blob/main/01_Plotly%E7%BB%98%E5%9B%BE/09_3D%E5%9B%BE%E8%A1%A8/04_%E4%B8%89%E7%BB%B4%E7%A9%BA%E9%97%B4%E5%9B%BE%E7%89%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 三维空间图片

在3D空间中，绘制图片，即将png/jpg等图片，绘制到3D Charts中。

# 调色板

色域空间有`1`、`L`、`P`、`RGB`、`RGBA`、`CMYK`、`YCbCr`、`LAB`、`HSV`、`I`、`F`等（在python pillow中称为[Mode](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes)）。 其中色域`P`采用调色板方式进行着色，调色板有`WEB`和`ADAPTIVE`两种。因为Plotly前端显示采用的是Javascript，属于web显示，所以调色板采用WEB调色板。首先，我们来看什么是调色板：

```python
[[0, 'rgb(0,0,255)'], [1, 'rgb(0,255,0)'], [2, 'rgb(255,0,0)']]
```

上面是一个简单的调色板，只能表示3种颜色：`0`、`1`、`2` ，依次对应RGB颜色中的`(0, 0, 255)`、`(0, 255, 0)`、`(255, 0, 0)`。显然，这个调色板太小，无法表示所有的RGB颜色，我们可以弄个大点的：

```python
[[0, 'rgb(0,0,0)'], [1, 'rgb(0,0,1)'], ..., [255*255*255, 'rgb(255,255,255)']]
```

显然，上面这个调色板，足够大了，可以表示所有RGB颜色，使用这个调色板，色域RGB转到色域P，是无损转换。**在实际应用中，为了效率，我们往往不会创建一个`0~255*255*255`这么大的调色板，而是回选择一个小一点的，比如`0~255`就可以了。** 下面演示如何获取调色板：

In [4]:
import numpy as np
from PIL import Image

# 首先，创建一个空白图片
img = np.ones((3,3,3), dtype='uint8')
img = Image.fromarray(img)
# 从RGB色域，转为P色域，并采用WEB调色板
img = img.convert('P', palette='WEB')
# 获取调色板
palette = img.getpalette()
palette = np.array(palette).reshape((-1, 3))
# palette记录了：P色域的0~255值，依次对应的RGB值
print(palette.shape)

(256, 3)


此外，Plotly使用的调色板，使用了值 `0~1` 的小数来表示：

```python
# 注意：实际中1不对应rgb(255, 255, 255),而是其它RGB值。这里只是为了演示方便，其它值亦然。
[[0, 'rgb(0,0,0)'], [0.1, 'rgb(0,0,1)'], ..., [1, 'rgb(255,255,255)']]
```

因此，使用`Image.convert('P', palette='WEB')`转换的图片，要显示出来，需要同时将图片和调色板，进行归一化：

```python
# 注意：实际中1不对应rgb(255, 255, 255),而是其它RGB值。这里只是为了演示方便，其它值亦然。
[[0, 'rgb(0,0,0)'], [1/255, 'rgb(0,0,1)'], ..., [255/255, 'rgb(255,255,255)']]

# 图片归一化，这一步一会不用做，一会让plotly.graph_objects.Surface()
# 内部自动完成即可。自动完成的前提，要通过`Surface(cmin=0, cmin=255)`参数，传入
# 调色板的颜色范围为0~255。
image = image / 255
```

上面，我们通过 `getpalette()` 方法，获得 `P(WEB) -> RGB` 的转换方式。下面，使用 `convert()` 方法，实现 `RGB -> P(WEB)` 转换：

In [5]:
import numpy as np
from PIL import Image


image = np.array([[[10, 11, 12], [13, 14, 15], [16, 17, 18], [16, 17, 18], [16, 17, 18]],
          [[20, 21, 22], [23, 24, 25], [26, 27, 28], [23, 24, 25], [26, 27, 28]],
          [[30, 31, 32], [33, 34, 35], [36, 37, 38], [33, 34, 35], [36, 37, 38]],
          [[40, 41, 42], [43, 44, 45], [46, 47, 48], [43, 44, 45], [46, 47, 48]]], dtype='uint8')
# 创建一张RGB图片
rgb_image = Image.fromarray(image)
# 从RGB色域转为P色域
web_image = rgb_image.convert(mode='P', palette='WEB')
# 从中可以看出，这个转换是有损转换
print(np.array(rgb_image).shape)
print(np.array(web_image).shape)
print(np.array(web_image))

(4, 5, 3)
(4, 5)
[[ 0  0  0 46 16]
 [46 17 53 11 52]
 [17 46 53 52 11]
 [53 53 53 53 53]]


# 绘制平面

在`3D平面`中，我们之前演示过如何使用各种方式绘制平面。根据图片特点，我们选择以下方式，绘制图像的平面：

In [6]:
import numpy as np
import plotly.graph_objects as go

x = np.array([6, 7], dtype=np.float64)
y = np.array([8, 9], dtype=np.float64)
z = np.array([[2, 3], 
        [4, 5]], dtype=np.float64)
# 设置颜色
surfacecolor = np.array([[0.0, 1.1], [2.2, 3.3]], dtype=np.float64)
# [(6, 8, 2) (6, 9, 4) (7, 8, 3) (7, 9, 5)]
data = go.Surface(z=z, x=x, y=y, surfacecolor=surfacecolor)

fig = go.Figure(data=data)
fig.show()

# 绘制图片

利用上面知识，实现在3D三维平面绘制图片：

In [7]:
import plotly.express as px
from PIL import Image
from scipy import misc

image = misc.face()
image = Image.fromarray(image)
image = image.resize((256, 200))
fig = px.imshow(image)
fig.show()

## 水平绘制

Z轴为恒定值，图片和X-Y平面平行：

In [8]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image
from scipy import misc


# 读取图片
image = misc.face()

# RGB色域转为P色域
image = Image.fromarray(image).convert('P', palette='WEB', dither=None)
image = image.resize((256, 200))
# 获得调色板
palette = np.array(image.getpalette()).reshape((-1, 3))
# Surface的colorscale格式为[i, 'rgb(R, G, B)']，i的范围为0~1.0，因此调色板从0~255归一化到0.0~1.0之间
colorscale = [[i/255.0, "rgb({}, {}, {})".format(*rgb)] for i, rgb in enumerate(palette)]

# 图片像素空间分布
width_x, height_y, = image.size
x = np.linspace(0, width_x, width_x)
y = np.linspace(0, height_y, height_y)
z = np.zeros((height_y, width_x))

# P色域的image的值的范围为0~255，所以cmin=0，cmax=255
data = go.Surface(x=x, y=y, z=z, surfacecolor=image, cmin=0, cmax=255, colorscale=colorscale)
fig = go.Figure(data=data)
fig.show()